In [1]:
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install requests

import numpy as np
import re
import requests
import pandas as pd
import seaborn as sns
from functools import reduce

# Igel
from igel import Igel

# Compare Algorithms
import matplotlib.pyplot as plt
from matplotlib import rcParams

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def clean_dataset(df):
    #assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    #df.dropna(inplace=True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df
    #return df[indices_to_keep].astype(np.float64)
    

def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

In [3]:
priority_scorer = {
    '10' : 'Critical',
    '9' : 'Critical',
    '8' : 'Critical',
    '7' : 'High',
    '6' : 'High',
    '5' : 'Medium',
    '4' : 'Medium',
    '3' : 'Low',
    '2' : 'Low',
    '1.0' : 'Low',
    '0.0' : 'Low',
}

priority_mapping = {
    "Critical" : 10,
    "High" : 7.5,
    "Medium" : 5,
    "Low" : 2.5,
    "Unknown" : 0,
}

# What we consider to be highly important categories of information
highCategoriser = {
    0.0 : 'Other-Advice',
    1.0 : 'Other-Advice',
    2.0 : 'Report-CleanUp',
    3.0 : 'ContextualInformation',
    4.0 : 'Other-ContextualInformation',
    5.0 : 'CallToAction-Donations',
    6.0 : 'Report-EmergingThreats',
    7.0 : 'Report-Factoid',
    8.0 : 'Report-FirstPartyObservation',
    9.0 : 'Request-GoodsServices',
    10.0 : 'Report-Hashtags',
    11.0 : 'Request-InformationWanted',
    12.0 : 'Other-Irrelevant',
    13.0 : 'Report-Location',
    14.0 : 'CallToAction-MovePeople',
    15.0 : 'Report-MultimediaShare',
    16.0 : 'Report-NewSubEvent',
    17.0 : 'Report-News',
    18.0 : 'Report-Official',
    19.0 : 'Report-OriginalEvent',
    20.0 : 'Request-SearchAndRescue',
    21.0 : 'Other-Sentiment',
    22.0 : 'Report-ServiceAvailable',
    23.0 : 'Report-ThirdPartyObservation',
    24.0 : 'CallToAction-Volunteer',
    25.0 : 'Report-Weather',
    26.0 : 'hmm',
}

event_int_map =	{
  "guatemalaEarthquake2012": 7,
  "joplinTornado2011": 16,
  "athensEarthquake2020": 35,
  "baltimoreFlashFlood2020": 36,
  "brooklynBlockPartyShooting2020": 37,
  "daytonOhioShooting2020": 38,
  "elPasoWalmartShooting2020": 39,
  "gilroygarlicShooting2020": 40,
  "hurricaneBarry2020": 41,
  "indonesiaEarthquake2020": 42,
  "keralaFloods2020": 43,
  "myanmarFloods2020": 44,
  "papuaNewguineaEarthquake2020": 45,
  "siberianWildfires2020": 46,
  "typhoonKrosa2020": 47,
  "typhoonLekima2020": 48,
  "whaleyBridgeCollapse2020": 49
}

mymap = {'Advice':1, 'CleanUp':2, 'ContextualInformation':3, 'Discussion':4, 'Donations':5, 
        'EmergingThreats':6, 'Factoid':7, 'FirstPartyObservation':8, 'GoodsServices':9, 'Hashtags':10, 
        'InformationWanted':11,'Irrelevant':12, 'Location':13, 'MovePeople':14, 
         'MultimediaShare':15, 'NewSubEvent':16, 'News':17,
        'Official':18, 'OriginalEvent':19, 'SearchAndRescue':20, 'Sentiment':21, 'ServiceAvailable':22, 
         'ThirdPartyObservation':23,'Volunteer':24, 'Weather':25}

## Feature Vector

Load the feature vector in from Play

In [4]:
#url = 'http://localhost:9000/stored_tweets'
url = 'http://tweetminer-2336003gproject.ida.dcs.gla.ac.uk/stored_tweets'

data = requests.get(url).json()

In [5]:
print(data)

{'fileName': 'data/tweets/run/gil.jsonl', 'tweets': [{'tweet_id': 1155907432607932419, 'tweet_text': '[shooter, gilroy, garlic, festival]', 'features': {'numb_of_urls': None, 'numb_of_hashtags': 0.0, 'numb_of_personal_pronouns': 0.0, 'numb_of_present_tenses': 0.0, 'weighted_length': 61.0, 'permillage': 217.0, 'tweet_created_at': 1564424782000.0, 'tweet_id_str': 1.1559074326079324e+18, 'positive_sentiment': 0.0, 'negative_sentiment': 0.0, 'numb_of_mentions': 0.0, 'numb_of_media': 1.0, 'numb_of_past_tenses': 0.0, 'numb_of_weird_chars': 8.0, 'numb_of_questions': 0.0, 'numb_of_emoticons': 0.0, 'numb_of_swearing_word': 0.0, 'numb_of_slang_words': 0.0, 'numb_of_intensifiers': 0.0, 'tweet_length': 35.0, 'userFollowersCount': 214.0, 'userFriendsCount': 233.0, 'user_numb_of_tweets': 0.0, 'user_list_count': 0.0, 'dict_precision': 0.0, 'dict_recall': 0.0, 'dict_f_measure': 0.0, 'offset': 0.0, 'is_verified': 0.0}, 'tfIdf': {'garlic': 0.096915126, 'festival': 0.12904063, 'shooter': 0.63826114, 'gil

In [6]:
df = pd.DataFrame(data)

In [7]:
df = pd.json_normalize(df['tweets'])

In [8]:
#df['sentiment']

In [9]:
df['sentiment'] = df[df['sentiment'].apply(lambda x:pd.Series(x))]
df_sentiment=pd.concat([df['sentiment'].str.split(',', expand=True)], axis=1, keys="s")
df_sentiment

INFO - NumExpr defaulting to 8 threads.


,s
,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
496,NaN
497,NaN
498,NaN


In [10]:
df.join(df_sentiment)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfId

In [11]:
df['embeddings'] 

0      {0.476523,0.27556002,-2.2285397,1.3045999,2.46...
1      {0.186423,1.67845,-3.56014,1.20671,2.63286,0.7...
2      {0.002332,0.17932999,-1.26695,0.65498996,2.328...
3      {0.013441,0.23682,-0.16899,0.40951,0.63812,0.4...
4      {2.3145852,1.3292699,-2.9821076,1.5878001,2.93...
                             ...                        
496    {1.0967231,0.32357,-0.55388004,2.03992,1.08545...
497    {0.013441,0.23682,-0.16899,0.40951,0.63812,0.4...
498    {1.965052,2.88709,-3.123487,1.0144439,6.922600...
499    {-0.86926794,-0.045750007,1.02722,-2.0003319E-...
500    {-0.5185759,1.4468,-2.01003,0.40061998,3.22344...
Name: embeddings, Length: 501, dtype: object

In [12]:
df['embeddings'] = df['embeddings'].str.extract('(\d+)', expand=False)
df['embeddings'] = df['embeddings'].str[1:]
df['embeddings']

0       
1       
2       
3       
4       
      ..
496     
497     
498     
499     
500     
Name: embeddings, Length: 501, dtype: object

In [13]:
df_embeddings = pd.concat([df['embeddings'].str.split(',', expand=True)], axis=1, keys="e")
df_embeddings

,e
,0
0,
1,
2,
3,
4,
...,...
496,
497,
498,


In [14]:
df.join(df_embeddings)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfId

In [15]:
df_emb_sent=df_sentiment.join(df_embeddings)
df_emb_sent

,s,e
,0,0
0,NaN,
1,NaN,
2,NaN,
3,NaN,
4,NaN,
...,...,...
496,NaN,
497,NaN,
498,NaN,


In [16]:
df.join(df_emb_sent)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfId

In [17]:
# Export
df.to_csv("temp/api_input.csv", index=False)


In [18]:
api_input = pd.read_csv("temp/api_input.csv")
api_input

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfId

In [19]:
# Drop the string categories
api_input.drop(['tweet_text'], axis = 1, inplace = True)
api_input.drop(['embeddings'], axis = 1, inplace = True)
api_input.drop(['sentiment'], axis = 1, inplace = True)

api_input.describe()

,tweet_id,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfIdf.reform,tfIdf.loved,tfIdf.actio

In [20]:
# Group by tweet
feature_vector_input = api_input.groupby(['tweet_id']).agg('first')
feature_vector_input

,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfIdf.reform,tfIdf.loved,tfIdf.action,tfIdf.e

## Load the labelled data

These are generated in 0_Labels.ipynb

In [21]:
# dtypes needs to be specified or long ints will change
labels_df = pd.read_json("../../../data/input/raw/data/2020/2020-A/labels/TRECIS-2018-2020A-labels.json", dtype={} )

# replace the event with a numeric value
labels_df = labels_df.replace({'eventID': event_int_map})

# Count the number of labels
labels_df['num'] = labels_df['postCategories'].str.len()


# Map the priority to a numeric value
labels_df = labels_df.replace({"postPriority": priority_mapping})

# Split categories and map to numeric values
category_list = pd.DataFrame(labels_df["postCategories"].to_list(), columns=['cat1', 'cat2', 'cat3',
                                                                   'cat4', 'cat5', 'cat6',
                                                                   'cat7', 'cat8', 'cat9', 'cat10'])


# Map the categories to numeric values
category_list = category_list.applymap(lambda s: mymap.get(s) if s in mymap else s)


# Join back onto our original list
labels = labels_df.join(category_list)

# Drop the string categories
labels.drop(['postCategories'], axis = 1, inplace = True)

# Tidy
labels = labels.drop(['eventName', 'eventDescription', 'eventType'], axis=1)


# Fill the NaN slots with 0
labels = labels.fillna("0")

# Export
labels.to_csv("../labels.csv", index=False)
labels = clean_dataset_int(labels)
#labels = clean_dataset_new(labels)

labels

,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10
0,16.0,7.267628e+16,2.5,3.0,7.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.0,7.267840e+16,10.0,4.0,22.0,18.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16.0,7.268240e+16,2.5,2.0,21.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,7.269393e+16,5.0,3.0,23.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.0,7.269856e+16,2.5,3.0,23.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42946,47.0,1.162000e+18,2.5,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42947,47.0,1.162005e+18,2.5,3.0,13.0,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42948,47.0,1.162005e+18,2.5,2.0,13.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42949,47.0,1.162006e+18,2.5,3.0,13.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train.csv


Merges the input feature vector with the labels

In [22]:
train = pd.merge(labels, feature_vector_input, left_on = 'postID', right_on = 'tweet_id', how = 'inner')

train = train.reset_index()
train.fillna(0, inplace=True)

train.replace([np.inf, -np.inf], np.nan, inplace=True)

train = train.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

train.to_csv("../train.csv", index=False)

train

,index,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,...,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfIdf.reform,tfIdf.loved,tfIdf.act

# Test.csv


Drops the categories, number of categories and priority so we can make our prediction

In [23]:
test = train

# Drop cat*
test.drop(list(test.filter(regex = 'cat\d+$')), axis = 1, inplace = True)

# Drop priority / num (of labels)
test.drop(['postPriority', 'num'], axis = 1, inplace = True)

# export
test.to_csv("../test.csv", index=False)

test

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,tfIdf.times,tfIdf.public,tfIdf.owned,tfIdf.sportsmanship,tfIdf.hunting,tfIdf.breaking,tfIdf.rifle,tfIdf.mad,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfIdf.reform,tfIdf.loved

# Igel

This notebook implements `Igel`

> Igel supports all sklearn's machine learning functionality,

Igel's supported models:

        +--------------------+----------------------------+-------------------------+
        |      regression    |        classification      |        clustering       |
        +--------------------+----------------------------+-------------------------+
        |   LinearRegression |         LogisticRegression |                  KMeans |
        |              Lasso |                      Ridge |     AffinityPropagation |
        |          LassoLars |               DecisionTree |                   Birch |
        | BayesianRegression |                  ExtraTree | AgglomerativeClustering |
        |    HuberRegression |               RandomForest |    FeatureAgglomeration |
        |              Ridge |                 ExtraTrees |                  DBSCAN |
        |  PoissonRegression |                        SVM |         MiniBatchKMeans |
        |      ARDRegression |                  LinearSVM |    SpectralBiclustering |
        |  TweedieRegression |                      NuSVM |    SpectralCoclustering |
        | TheilSenRegression |            NearestNeighbor |      SpectralClustering |
        |    GammaRegression |              NeuralNetwork |               MeanShift |
        |   RANSACRegression | PassiveAgressiveClassifier |                  OPTICS |
        |       DecisionTree |                 Perceptron |                    ---- |
        |          ExtraTree |               BernoulliRBM |                    ---- |
        |       RandomForest |           BoltzmannMachine |                    ---- |
        |         ExtraTrees |       CalibratedClassifier |                    ---- |
        |                SVM |                   Adaboost |                    ---- |
        |          LinearSVM |                    Bagging |                    ---- |
        |              NuSVM |           GradientBoosting |                    ---- |
        |    NearestNeighbor |        BernoulliNaiveBayes |                    ---- |
        |      NeuralNetwork |      CategoricalNaiveBayes |                    ---- |
        |         ElasticNet |       ComplementNaiveBayes |                    ---- |
        |       BernoulliRBM |         GaussianNaiveBayes |                    ---- |
        |   BoltzmannMachine |      MultinomialNaiveBayes |                    ---- |
        |           Adaboost |                       ---- |                    ---- |
        |            Bagging |                       ---- |                    ---- |
        |   GradientBoosting |                       ---- |                    ---- |
        +--------------------+----------------------------+-------------------------+

In [24]:
params = {
        'cmd': 'fit',    
        'data_path': "../train.csv",
        'yaml_path': 'yaml/multi.yaml'  # DecisionTree
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'fit', 'data_path': '../train.csv', 'yaml_path': 'yaml/multi.yaml'}
INFO - Executing command: fit ...
INFO - reading data from ../train.csv
INFO - You passed the configurations as a yaml file.
INFO - your chosen configuration: {'dataset': {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}}, 'model': {'type': 'regression', 'algorithm': 'DecisionTree'}, 'target': ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']}
INFO - dataset_props: {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}} 
model_props: {'type': 'regression', 'algorithm': 'DecisionTree'} 
 target: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num'] 

INFO - dataset shape: (271, 506)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'postPr

INFO - y shape: (271, 12) and x shape: (271, 494)
INFO - Solving a regression problem using ===> DecisionTree
INFO - model arguments: 
None
INFO - executing a DecisionTreeRegressor algorithm...
INFO - predicting multiple targets detected. Hence, the model will be automatically converted to a multioutput model
INFO - cross validation is not provided
INFO - Folder /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results already exists
WARNING - data in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder will be overridden. If you don't want this, then move the current /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results to another path
INFO - Successfully created the directory in /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results 
INFO - model saved successfully and can be found in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder
INFO - split option detected. The performance will be a

In [25]:
params = {
        'cmd': 'predict',    
        'data_path': "../test.csv",
        'yaml_path': 'yaml/hyper.yaml'
}
Igel(**params)

INFO - Entered CLI args: {'cmd': 'predict', 'data_path': '../test.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: predict ...
INFO - reading data from ../test.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - loading from /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - dataset shape: (271, 494)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'offset', 'features.numb_of_urls', 'features.numb_of_hashtags', 'features.numb_of_personal_pronouns', 'features.numb_of_present_tenses', 'features.weighted_length', 'features.permillage', 'features.tweet_created_at', 'features.tweet_id_str', 'features.positive_sentiment', 'features.negative_sentiment', 'features.numb_of_mentions', 'features.numb_of_media', 'features.numb_of_past_tenses', 'features.numb_of_weird_chars', 'features.numb_of_questions', 'features.numb_of_emoticons', 'features.numb_of_swearing_wor

INFO - predictions shape: (271, 12) | shape len: 2
INFO - predict on targets: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']
INFO - saving the predictions to /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/predictions.csv


# Predictions

1. View the raw predictions
2. Map the labels to their High Level Information Types
3. Merge the predictions back into the training set


In [26]:

def is_neg_predictions(predictions):
    predictions = predictions.sort_values(by=['postPriority'])
    predictions = predictions[(predictions > 0).all(1)]
    predictions.round()

    
predictions = pd.read_csv("model_results/predictions.csv")
predictions

#is_neg_predictions(predictions)

,postPriority,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,num
0,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
266,2.5,23.0,10.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
267,2.5,23.0,10.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
268,5.0,11.0,15.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
269,2.5,23.0,10.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


### Merge the new predictions back onto dataframe with the missing columns

In [27]:
# Map the labels to their High Level Information Types
cat_list = predictions.filter(regex='cat', axis=1).round().applymap(lambda x: highCategoriser[x])

#
predictions = cat_list.combine_first(predictions)

# Merge the predictions back into the training set
df = test.merge(predictions, left_index=True, right_index=True)


# Append the predicted categories to a list in a new column
df['predicted_categories'] = df[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10']].values.tolist()


# Get the number of categories into something we can use to index
df['num'] = df['num'].astype(float).astype(int)

# Remove categories beyond what the tweet is predicted to have
df['categories'] = df.apply(lambda x: x['predicted_categories'][0:x['num']], axis=1)


df

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.garlic,tfIdf.festival,tfIdf.shooter,tfIdf.gilroy,tfIdf.shooting,tfIdf.prayers,tfIdf.frl,tfIdf.buggin,tfIdf.smh,tfIdf.ughhhh,tfIdf.chicken,tfIdf.aaaaaaaahhhh,tfIdf.inasal,tfIdf.rice,tfIdf.kevon,tfIdf.shot,tfIdf.vampires,tfIdf.nat,tfIdf.iranian,tfIdf.white,tfIdf.italian,tfIdf.fuck,tfIdf.shoot,tfIdf.sacrifices,tfIdf.wounded,tfIdf.guncontrolnow,tfIdf.lives,tfIdf.fed,tfIdf.nra,tfIdf.boy,tfIdf.god,tfIdf.blood,tfIdf.california,tfIdf.killed,tfIdf.food,tfIdf.petition,tfIdf.stopthenra,tfIdf.emergency,tfIdf.gilroyshooting,tfIdf.enoughisenough,tfIdf.mass,tfIdf.doesn,tfIdf.bread,tfIdf.victims,tfIdf.romero,tfIdf.stephen,tfIdf.deserved,tfIdf.failed,tfIdf.oneplateau,tfIdf.america,tfIdf.luciano,tfIdf.cranberries,tfIdf.dndharvesting,tfIdf.black,tfIdf.mulberry,tfIdf.arabica,tfIdf.coffe,tfIdf.lowlife,tfIdf.shoots,tfIdf.update,tfIdf.happening,tfIdf.authorities,tfIdf.death,tfIdf.mcdonald,tfIdf.relate,tfIdf.ftw,tfIdf.chilli,tfIdf.gun,tfIdf.gilroygarlicfestival,tfIdf.control,tfIdf.safe,tfIdf.stay,tfIdf.hell,tfIdf.police,tfIdf.lorene,tfIdf.accomplice,tfIdf.hunt,tfIdf.mystery,tfIdf.motive,tfIdf.read,tfIdf.kills,tfIdf.idiotic,tfIdf.question,tfIdf.insane,tfIdf.broccoli,tfIdf.roasted,tfIdf.carrots,tfIdf.changing,tfIdf.reported,tfIdf.son,tfIdf.father,tfIdf.live,tfIdf.guns,tfIdf.wtf,tfIdf.livers,tfIdf.tots,tfIdf.lizard,tfIdf.didn,tfIdf.trump,tfIdf.mention,tfIdf.morning,tfIdf.wasn,tfIdf.children,tfIdf.lost,tfIdf.racist,tfIdf.family,tfIdf.brand,tfIdf.injured,tfIdf.watch,tfIdf.officials,tfIdf.liveonk,tfIdf.gilroyactiveshooter,tfIdf.updates,tfIdf.gt,tfIdf.situation,tfIdf.swordfish,tfIdf.butter,tfIdf.add,tfIdf.eat,tfIdf.owner,tfIdf.pretty,tfIdf.imagine,tfIdf.vet,tfIdf.rural,tfIdf.combat,tfIdf.partner,tfIdf.cali,tfIdf.folks,tfIdf.chuck,tfIdf.weapons,tfIdf.prayforthevictims,tfIdf.santino,tfIdf.identified,tfIdf.william,tfIdf.legan,tfIdf.fucking,tfIdf.upset,tfIdf.active,tfIdf.grease,tfIdf.prepare,tfIdf.shrimp,tfIdf.pleasant,tfIdf.noodles,tfIdf.taste,tfIdf.noodle,tfIdf.girl,tfIdf.cute,tfIdf.picture,tfIdf.mac,tfIdf.worth,tfIdf.cheese,tfIdf.crab,tfIdf.deadly,tfIdf.haley,tfIdf.vampire,tfIdf.condolences,tfIdf.community,tfIdf.onion,tfIdf.lime,tfIdf.cloves,tfIdf.tomato,tfIdf.juice,tfIdf.salsa,tfIdf.red,tfIdf.tomatoes,tfIdf.shook,tfIdf.flakes,tfIdf.coriander,tfIdf.zavier,tfIdf.soft,tfIdf.infection,tfIdf.spinach,tfIdf.disease,tfIdf.oranges,tfIdf.foods,tfIdf.amp,tfIdf.kefir,tfIdf.blueberries,tfIdf.immunesystem,tfIdf.oilyfish,tfIdf.almonds,tfIdf.darkchocolate,tfIdf.help,tfIdf.redbellpeppers,tfIdf.sunflowerseeds,tfIdf.kiwis,tfIdf.greentea,tfIdf.ginger,tfIdf.strengthen,tfIdf.sweetpotatoes,tfIdf.turmeric,tfIdf.fight,tfIdf.reports,tfIdf.respond,tfIdf.witness,tfIdf.gunshots,tfIdf.fireworks,tfIdf.news,tfIdf.wtff,tfIdf.center,tfIdf.investigating,tfIdf.department,tfIdf.stand,tfIdf.command,tfIdf.jpost,tfIdf.dying,tfIdf.calls,tfIdf.husband,tfIdf.mom,tfIdf.whatsoever,tfIdf.accessible,tfIdf.assault,tfIdf.couple,tfIdf.unacceptable,tfIdf.ethical,tfIdf.grew,tfIdf.heart,...,tfIdf.bitch,tfIdf.named,tfIdf.federaldeathpenalty,tfIdf.broken,tfIdf.normal,tfIdf.laws,tfIdf.lose,tfIdf.real,tfIdf.families,tfIdf.reform,tfIdf.loved,tfIdf.action,tfIdf.evening,tfIdf.starting,tfIdf.violence,tfIdf.fresh,tfIdf.dead,tfIdf.suspect,tfIdf.mul

## Export

Export in the TRECIS format

In [28]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df.drop_duplicates(subset="postID").itertuples():
        #print("row:", row)
        content = [
            "TRECIS-CTIT-H-Test-0" + str(int(row.eventID)),
            "Q0",          
            np.int64(row.postID),   
            getattr(row, 'Index'),  #ToDo: Fix?
            #row.priority,
            str(priority_scorer[str(round(row.postPriority))[:3]]),  #ToDo: Fix
            row.categories,
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")